In [ ]:
# ---- 1. Import standard packages ----
import pandas as pd        # data handling
import numpy as np         # (optional) numerical helpers
import os                  # to list / verify files in the working dir
from IPython.display import display  # nicer display in notebooks

In [ ]:
# ---- 2. Read the CSV into a DataFrame ----
csv_path = "diabetic_data.csv"      # <- change if your file name differs
df = pd.read_csv(csv_path)

# ---- 3. Inspect the imported dataset ----
print("\nShape of dataset (rows, columns):", df.shape)

print("\nColumn names:")
print(df.columns.tolist())

print("\nFirst 5 rows:")
display(df.head())

# Optional: get a concise info summary
# print("\nDataFrame info():")
# df.info()

In [ ]:
# ---- 4. Create a column subset & independent copy ----
cols_wanted = [
    'patient_nbr', 'race', 'gender', 'age', 'admission_type_id',
    'num_lab_procedures', 'num_medications',
    'diag_1', 'diag_2',
    'metformin', 'insulin',
    'metformin-rosiglitazone', 'metformin-pioglitazone',
    'diabetesMed', 'readmitted'
]

# 1) Verify all requested columns exist in the original DataFrame
missing = [c for c in cols_wanted if c not in df.columns]
if missing:
    raise KeyError(f"The following requested columns are not present in the DataFrame: {missing}")

# 2) Create a true copy (so further edits won’t affect df)
subset_df = df[cols_wanted].copy()

# 3) Quick look
print("Subset shape:", subset_df.shape)
display(subset_df.head())


In [ ]:
# ---- 6. Rename the selected columns ----
rename_map = {
    'patient_nbr':             'patient_number',
    'admission_type_id':       'admission_type',
    'diag_1':                  'diagnoses_1',
    'diag_2':                  'diagnoses_2',
    'metformin-rosiglitazone': 'metformin_rosiglitazone',
    'metformin-pioglitazone':  'metformin_pioglitazone'
}

subset_df.rename(columns=rename_map, inplace=True)

# quick check
print("Columns after renaming:")
print(subset_df.columns.tolist())
display(subset_df.head())


In [ ]:
# ---------------------------------------------------------------
#  Detect and segregate rows with problematic values in two fields:
#    • gender            – entries listed as 'Unknown/Invalid'
#    • diagnoses_1       – NaN, empty, '?', 'UNK', 'UNKNOWN', etc.
# ---------------------------------------------------------------

# 1) Flag rows where gender is 'Unknown/Invalid'
mask_unknown_gender = subset_df['gender'].str.strip().str.upper() == 'UNKNOWN/INVALID'

# 2) Flag rows where diagnoses_1 is missing / invalid
mask_missing_diag1 = (
    subset_df['diagnoses_1'].isna() |                              # genuine NaN
    subset_df['diagnoses_1'].str.strip().eq('') |                  # empty string
    subset_df['diagnoses_1'].str.strip().str.upper().isin(         # placeholders
        {'?', 'UNK', 'UNKNOWN'}
    )
)

# ---------------------------------------------------------------
#  Create convenience DataFrames
# ---------------------------------------------------------------

# Rows with unknown/invalid gender
unknown_gender_df = subset_df.loc[mask_unknown_gender].copy()

# Rows with missing/invalid primary diagnosis
missing_diag1_df  = subset_df.loc[mask_missing_diag1].copy()

# Clean dataset with both problems removed
clean_df = subset_df.loc[~(mask_unknown_gender | mask_missing_diag1)].copy()

# ---------------------------------------------------------------
#  Quick counts
# ---------------------------------------------------------------
print(f"Unknown/Invalid gender rows      : {len(unknown_gender_df)}")
print(f"Missing/invalid diagnoses_1 rows : {len(missing_diag1_df)}")
print(f"Rows retained for analysis       : {len(clean_df)} out of {len(subset_df)}")

# (Optional) peek at the problematic rows
# unknown_gender_df.head()
# missing_diag1_df.head()

# (Optional) save or log the subsets
# unknown_gender_df.to_csv('rows_unknown_gender.csv', index=False)
# missing_diag1_df.to_csv('rows_missing_diag1.csv',  index=False)
# clean_df.to_csv('subset_clean.csv', index=False)


In [ ]:
# ---------------------------------------------------------------
#  Remove rows with problematic gender OR primary-diagnosis values
# ---------------------------------------------------------------

# 1) Build the same Boolean masks
mask_unknown_gender = subset_df['gender'].str.strip().str.upper() == 'UNKNOWN/INVALID'
mask_missing_diag1  = (
    subset_df['diagnoses_1'].isna() |
    subset_df['diagnoses_1'].str.strip().eq('') |
    subset_df['diagnoses_1'].str.strip().str.upper().isin({'?', 'UNK', 'UNKNOWN'})
)

# 2) Combine masks: rows to drop
rows_to_drop = mask_unknown_gender | mask_missing_diag1

# 3) Drop them and create a new cleaned DataFrame
subset_clean = subset_df.loc[~rows_to_drop].copy()

# 4) Quick sanity-check
print(f"Rows dropped : {rows_to_drop.sum()}")          # expected 24 (3 + 21)
print(f"Rows kept    : {len(subset_clean)}")            # expected 101,742

# 5) (Optional) overwrite the original variable or save to disk
# subset_df = subset_clean           # uncomment if you want to re-use the same name
# subset_clean.to_csv('subset_clean.csv', index=False)


In [ ]:
# ----------------------------------------------------
# Verify that gender and diagnoses_1 are now clean
# ----------------------------------------------------

# 1) Gender check  ─ any Unknown/Invalid left?
remaining_bad_gender = subset_clean['gender'].str.strip().str.upper().eq('UNKNOWN/INVALID').sum()

# 2) diagnoses_1 check ─ any null / blank / placeholders left?
remaining_bad_diag1 = (
    subset_clean['diagnoses_1'].isna() |
    subset_clean['diagnoses_1'].str.strip().eq('') |
    subset_clean['diagnoses_1'].str.strip().str.upper().isin({'?', 'UNK', 'UNKNOWN'})
).sum()

print(f"Remaining Unknown/Invalid gender rows   : {remaining_bad_gender}")
print(f"Remaining missing/invalid diagnoses_1   : {remaining_bad_diag1}")


In [ ]:
# ----------------------------------------------------
# Inspect the 'race' column for missing / invalid values
# ----------------------------------------------------
invalid_tokens = {'?', 'UNK', 'UNKNOWN', 'INVALID'}   # extend if needed

missing_mask = (
    subset_clean['race'].isna() |                       # True NA (np.nan)
    subset_clean['race'].str.strip().eq('') |           # empty string
    subset_clean['race'].str.strip().str.upper().isin(invalid_tokens)
)

missing_race_count = missing_mask.sum()

print(f"Missing / invalid race rows : {missing_race_count} "
      f"out of {len(subset_clean)}")


In [ ]:
# --- show the offending rows ---------------------------------
problem_rows = subset_clean[missing_mask]          # filter by the mask
display(problem_rows)                              # pretty HTML table in Jupyter
# or, if you prefer plain print:
# print(problem_rows.to_string(max_rows=None))


In [ ]:
# ----------------------------------------------------
# Replace literal '?' in the race column with 'Missing'
# ----------------------------------------------------
subset_clean['race'] = (
    subset_clean['race']                 # original column
      .str.strip()                       # remove any leading / trailing spaces
      .replace({'?': 'Missing'})         # map ?  ->  Missing
)

# quick check
print(subset_clean['race'].value_counts(dropna=False))


In [ ]:
# ----------------------------------------------------
# Map admission_type codes ➜ human-readable labels
# ----------------------------------------------------
admission_map = {
    1: "Emergency",
    2: "Urgent",
    3: "Elective",
    4: "Newborn",
    5: "Not Available",
    6: "NULL",
    7: "Trauma Center",
    8: "Not Mapped"
}

# If the column is still string-typed, cast to numeric first (errors='coerce' keeps bad codes as NaN)
subset_clean['admission_type'] = (
    pd.to_numeric(subset_clean['admission_type'], errors='coerce')
      .map(admission_map)
      .fillna(subset_clean['admission_type'])   # fall back to original value if code wasn’t in the map
)

# ----------------------------------------------------
# Quick check + preview
# ----------------------------------------------------
print("Unique values after mapping:\n", subset_clean['admission_type'].value_counts(dropna=False), '\n')
display(subset_clean.head())        # or print(subset_clean) if you really want the whole frame


In [ ]:
# ----------------------------------------------------
# Check for missing / invalid values in the 'diagnoses_2' column
# ----------------------------------------------------
missing_diag2_mask = (
    subset_clean['diagnoses_2'].isna()                    |  # real NA / NaN
    subset_clean['diagnoses_2'].str.strip().eq('')        |  # empty string
    subset_clean['diagnoses_2'].str.strip().eq('?')       |  # question-mark placeholder
    subset_clean['diagnoses_2'].str.strip().str.upper().isin(
        {'UNK', 'UNKNOWN', 'INVALID'}                     # other obvious placeholders
    )
)

missing_diag2_count = missing_diag2_mask.sum()
print(f"Missing / invalid diagnoses_2 rows : {missing_diag2_count}")


In [ ]:
# ----------------------------------------------------
# Replace missing / invalid values in 'diagnoses_2'
# ----------------------------------------------------
# (Assumes missing_diag2_mask has already been defined in the previous cell.)
subset_clean.loc[missing_diag2_mask, 'diagnoses_2'] = 'Missing'

# Optional: quick confirmation
new_missing = subset_clean['diagnoses_2'].isna().sum() + \
              (subset_clean['diagnoses_2'].str.strip() == '').sum() + \
              (subset_clean['diagnoses_2'].str.strip().eq('?')).sum()
print(f"Remaining missing/invalid diagnoses_2 rows after replacement : {new_missing}")


In [ ]:
subset_clean

In [ ]:
# ----------------------------------------------------
# Remove the patient_number column
# ----------------------------------------------------
subset_clean = subset_clean.drop(columns=['patient_number'])

# Quick check
print(subset_clean.columns)


In [ ]:
# -------------------------------------------
# Re-order columns: put the six key columns first
# -------------------------------------------
first_cols = [
    'race', 'gender', 'age',
    'admission_type',
    'diagnoses_1', 'diagnoses_2'
]

# Keep the remaining columns in their current order
remaining_cols = [c for c in subset_clean.columns if c not in first_cols]

# Re-assemble the DataFrame
subset_clean = subset_clean[first_cols + remaining_cols]

# Confirm the new column order
print(subset_clean.columns.tolist())


In [ ]:
# ============================================================
# 0)  Import anything we need
# ============================================================
import pandas as pd               # already imported earlier, but harmless

# ============================================================
# 1)  Function: map a raw ICD-9 code ➜ group name
# ============================================================
def map_icd_to_group(code: str) -> str:
    """
    Convert any ICD-9 code string to one of the 9 allowed group names
        {Circulatory, Respiratory, Digestive, Diabetes, Injury,
         Musculoskeletal, Genitourinary, Neoplasms, Other, Missing}.
    Rules taken from Strack et al. (2014) table (screenshot).
    • 'Missing' rows are preserved as-is.
    • Non-numeric / E / V codes fall automatically into 'Other'.
    """
    # ---- 1. preserve Missing -------------------------------------------------
    if pd.isna(code) or str(code).strip().upper() == 'MISSING':
        return 'Missing'
    
    # ---- 2. grab leading 3-digit numeric prefix (ignore .xx) -----------------
    try:
        prefix = int(float(str(code)[:3]))          # ‘250.01’ ➜ 250
    except ValueError:                              # e.g. 'E878', 'V458'
        return 'Other'
    
    # ---- 3. apply the figure’s rules ----------------------------------------
    if (390 <= prefix <= 459) or prefix == 785:
        return 'Circulatory'
    if (460 <= prefix <= 519) or prefix == 786:
        return 'Respiratory'
    if (520 <= prefix <= 579) or prefix == 787:
        return 'Digestive'
    if prefix == 250:
        return 'Diabetes'
    if 800 <= prefix <= 999:
        return 'Injury'
    if 710 <= prefix <= 739:
        return 'Musculoskeletal'
    if (580 <= prefix <= 629) or prefix == 788:
        return 'Genitourinary'
    if 140 <= prefix <= 239:
        return 'Neoplasms'
    
    # All other ICD-9 blocks and single-code leftovers
    return 'Other'


# ============================================================
# 2)  Apply mapping to both diagnosis columns
# ============================================================
subset_clean = subset_clean.copy()     # keep a copy safe
for col in ['diagnoses_1', 'diagnoses_2']:
    subset_clean[col] = subset_clean[col].apply(map_icd_to_group)


# ============================================================
# 3)  Quick sanity-checks
# ============================================================

# 3a) confirm 'Missing' counts did NOT change
missing_counts = {col: (subset_clean[col] == 'Missing').sum()
                  for col in ['diagnoses_1', 'diagnoses_2']}
print(f"‘Missing’ retained  ➜  diag_1: {missing_counts['diagnoses_1']}, "
      f"diag_2: {missing_counts['diagnoses_2']}")

# 3b) list final value distribution for visual inspection
for col in ['diagnoses_1', 'diagnoses_2']:
    print(f"\nValue counts for {col}:")
    display(subset_clean[col].value_counts(dropna=False))


In [ ]:
# -------------------------------------------
# Save the final DataFrame in the notebook’s
# working directory (visible in “Files” pane)
# -------------------------------------------

# 1) Make an explicit copy (optional but clear)
diabetic_data_final = subset_clean.copy()

# 2) Choose a filename; no path needed → current dir
filename = "diabetic_data_final.csv"

# 3) Write the CSV
diabetic_data_final.to_csv(filename, index=False)

print(f"✔ Dataset saved as “{filename}” — {diabetic_data_final.shape[0]:,} rows, {diabetic_data_final.shape[1]} columns.")
